# OPER 610 - Brandon Hosley    Problem 1.2 (Phase 2)    Due Jan 17 2023

## Preparation

In [1]:
from pyomo.environ import * 
from pyomo.opt import SolverFactory

model = ConcreteModel()

## Variables and Coefficients

In [2]:
model.costs={'Compound1':20,'Compound2':30,'Compound3':20,'Compound4':15}
model.j=model.costs.keys()
model.compositions={'ElementA': {'Compound1':0.35,'Compound2':0.15,'Compound3':0.35,'Compound4':0.25},
                    'ElementB': {'Compound1':0.20,'Compound2':0.65,'Compound3':0.35,'Compound4':0.40},
                    'ElementC': {'Compound1':0.40,'Compound2':0.15,'Compound3':0.25,'Compound4':0.30}}
model.x = Var(model.j, domain = NonNegativeReals)

## Objective Function

In [5]:
model.objective = Objective(expr=sum(model.costs[j]*model.x[j] for j in model.j), sense=minimize)

## Constraints 

In [6]:
model.constraints=ConstraintList()
model.constraints.add(sum(model.x[j] for j in model.j) == 1)                               # Ensure Percentage sums to 1
model.constraints.add(sum(model.compositions['ElementA'][j]*model.x[j] for j in model.j) == 0.25) # Percentage Compound A
model.constraints.add(sum(model.compositions['ElementB'][j]*model.x[j] for j in model.j) >= 0.35) # Percentage Compound B
model.constraints.add(sum(model.compositions['ElementC'][j]*model.x[j] for j in model.j) >= 0.20) # Percentage Compound C
model.constraints.add(model.x['Compound1'] >= 0.25)
model.constraints.add(model.x['Compound2'] >= 0.30)

## Solve

In [7]:
solver = SolverFactory('glpk')
sol = solver.solve(model, report_timing=True, tee=True)

        0.01 seconds required to write file
        0.05 seconds required for presolve
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmp6uz6vqxe.glpk.raw
 --wglp /var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpsslh7100.glpk.glp
 --cpxlp /var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpbrdpo7d2.pyomo.lp
Reading problem data from '/var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpbrdpo7d2.pyomo.lp'...
7 rows, 5 columns, 19 non-zeros
56 lines were read
Writing problem data to '/var/folders/8h/pt6dnpv520d200zx4xc71ld80000gn/T/tmpsslh7100.glpk.glp'...
45 lines were written
GLPK Simplex Optimizer, v4.65
7 rows, 5 columns, 19 non-zeros
Preprocessing...
4 rows, 4 columns, 16 non-zeros
Scaling...
 A: min|aij| =  1.500e-01  max|aij| =  1.000e+00  ratio =  6.667e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 3
      0: obj =   2.075000000e+01 inf =  

## Results

In [9]:
print("The solver terminated with the following decision variable values:")              
for j in model.j:
    if(value(model.x[j])>0):
       print("  ", j, format(value(model.x[j]),".2f"))

The solver terminated with the following decision variable values:
   Compound1 0.30
   Compound2 0.30
   Compound4 0.40
